In [1]:
import pandas as pd
import random
import math
import numpy as np
import scipy.sparse.linalg
import matplotlib.pyplot as plt
import sklearn.metrics
import json
import csv
import networkx as nx

#Ease of Life libraries used to simulate progression of slow algorithms
from ipywidgets import IntProgress
from IPython.display import display
import time

In [2]:
#Load the data as given.
data = pd.read_csv("philly_users_businesses_stars.csv", encoding="latin")

print(data.info()) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 967552 entries, 0 to 967551
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   user_id      967552 non-null  object 
 1   business_id  967552 non-null  object 
 2   stars        967552 non-null  float64
dtypes: float64(1), object(2)
memory usage: 22.1+ MB
None


# Part 1: Aggressive Pruning
Firstly, we prune our data as needed, using a modified version of our code from the previous exercise set.

In [3]:
#STEP 1 (Same as STEP 1 of previous exercise 2.)
prevLen = -1
currentLen = len(data)
newData = data.copy()

#Iterate the copied data. If after the iteration the size is the same, stop. otherwise, repeat.
while prevLen != currentLen:
    print("Current Data Size: ", len(newData))
    
    #Prune Non-Active Users.
    activeUsers = newData.groupby(["user_id"]).count()
    activeUsers = activeUsers.loc[activeUsers["business_id"] > 30].reset_index()
    newData = newData.loc[data["user_id"].isin(activeUsers["user_id"].unique())]

    #Prune Non-Active Businesses.
    activeBusinesses = newData.groupby(["business_id"]).count()
    activeBusinesses = activeBusinesses.loc[activeBusinesses["user_id"] > 50].reset_index()
    newData = newData.loc[data["business_id"].isin(activeBusinesses["business_id"].unique())]
    
    prevLen = currentLen
    currentLen = len(newData)
print("\nTotal Unique Users: ", len(newData["user_id"].unique()))
print("Total Unique Businesses: ", len(newData["business_id"].unique()))
print("Total Size: ", len(newData))

Current Data Size:  967552
Current Data Size:  167056
Current Data Size:  117066
Current Data Size:  106727
Current Data Size:  103946
Current Data Size:  103070
Current Data Size:  102852
Current Data Size:  102472
Current Data Size:  102093
Current Data Size:  101923

Total Unique Users:  1604
Total Unique Businesses:  887
Total Size:  101923


In [52]:
#Load the data by chunk, finding the important data needed.
#<!-- Run this only once! We store the nodes for future iterations in filtered_users --!>
exit(1) #REMOVE TO RUN
chunkFrame = pd.DataFrame()
with pd.read_json("yelp_academic_dataset_user.json", lines=True, encoding="latin", chunksize=100_000) as file:
    for chunk in file:
        chunk = chunk[["user_id", "friends"]]
        chunk = chunk.loc[chunk["user_id"].isin(newData["user_id"])]
        user_df.append(chunk)
user_df.to_csv("filtered_users.csv", index=False, mode="w")

In [4]:
#Load the filtered users
userData = pd.read_csv("filtered_users.csv", encoding="latin")
#Turn the friends into an array
userData["friends"] = userData["friends"].apply(lambda x : x.split(", "))
#Remove those without friends
userData = userData.loc[~userData.index.isin(userData[userData.apply(lambda x : True if "None" in x.friends else False, axis=1)].index)]
reviewData = newData.copy()
print(userData.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1583 entries, 0 to 1603
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  1583 non-null   object
 1   friends  1583 non-null   object
dtypes: object(2)
memory usage: 37.1+ KB
None


# Part 1: The Graph
Having our data loaded, we find all users that we need, as well as their friendships within the same set of users. We get 1604 users, from which after finding the largest strongly connected component and isolating it, we end up with 1504 unique users, 887 businesses, and 19347 friendships amongst the users.

In [5]:
#Part 1 - Forge the graph
G = nx.Graph()

importantIDs = userData["user_id"].to_list()
#Initialize nodes
for uid in userData["user_id"]:
    G.add_node(uid)
#Connect edges
for uid, friends in zip(userData["user_id"], userData["friends"]):
    for f in friends:
        if f in importantIDs:
            G.add_edge(uid, f)
            
#Find the strongly connected components and remove the nodes not in them.
stronglyConnectedNodes = max(nx.connected_components(G), key=len)
removedNodes = [i for i in G.nodes if i not in stronglyConnectedNodes]
for node in removedNodes:
    G.remove_node(node)
    
#Prune unwanted users               
reviewData = reviewData.loc[reviewData["user_id"].isin(G.nodes())]
print("Users:", len(G.nodes), "\nFrienships:", len(G.edges), "\nBusinesses:", len(reviewData["business_id"].unique()))

Users: 1504 
Frienships: 19347 
Businesses: 887


# Part 2: Train And Test Data
A simple seperation of our data to sample them into train and test will suffice for this step. However, as requested, we instead load the pre-sampled data.

In [11]:
#Part 2
sampleUsers = random.sample(reviewData["user_id"].unique().tolist(), 100)
sampleBusinesses = random.sample(reviewData["business_id"].unique().tolist(), 100)

testData = reviewData.loc[reviewData["user_id"].isin(sampleUsers) & reviewData["business_id"].isin(sampleBusinesses)]
trainData = reviewData.loc[~reviewData.index.isin(testData.index)]

testData = pd.read_csv("test_data.csv", encoding="latin").drop("Unnamed: 0", axis=1).drop_duplicates()
trainData = pd.read_csv("train_data.csv", encoding="latin").drop("Unnamed: 0", axis=1).drop_duplicates()


# Part 3: Value Propagation
A somewhat similar approach as the PageRank algorithm. Before we start, we do some preproccessing on our graph:
All nodes get 2 attributes:
- "absorbing_bids" contains a list of all the business IDs that this user has in our train data. Therefore, we have a linear way of finding whether the pair U,B is absorbing.
- "r" contains a dictionary in the form of BID:R_VALUE. This represents the guess R_VALUE for the pair UID,BID where UID is the current node we are on. If no R value has been set, we have no entry on this dictionary.

The algorithm then begins as expected. For each node and for each business, we check if the pair of node and business is absorbant. If it is, we continue, otherwise, we calculate its new R_VALUE using its neighbors. All neighbors with no R_VALUE for that pair are just ignored. This continious until we see changes bellow our threshold (10^-6).

In [12]:
#Part 3
#Test BIDs
testBusinessIds = testData["business_id"].unique()

#A listed group-by dataframe of all UIDs in our train data.
absorbingPairsBids = trainData.groupby(["user_id"])["business_id"].apply(list).reset_index()
absorbingPairsStars = trainData.groupby(["user_id"])["stars"].apply(list).reset_index()
absorbingPairs = absorbingPairsBids.merge(absorbingPairsStars, right_on = ["user_id"], left_on = ["user_id"])

#Mark absorbant/not-absorbant
graph = G.copy()
for node in graph.nodes():
    bids = absorbingPairs[absorbingPairs["user_id"] == node]
    
    #Non Absorbant
    if len(bids) == 0:
        graph.nodes[node]["absorbing_bids"] = []
        graph.nodes[node]["r"] = {}
    else: #Absorbant
        graph.nodes[node]["absorbing_bids"] = bids.iloc[0]["business_id"]
        firstSample = bids.iloc[0]
        graph.nodes[node]["r"] = {firstSample["business_id"][i]:firstSample["stars"][i] for i in range(len(firstSample["stars"]))}
    

#absorbing_bids: All BIDs of this node that have a pair in the Train Data
#r: The R values. Default = null. If absorbant: A Dictionary matching Bid:Stars for the static values

In [8]:
hasBigChange = True
counter = 0

while (hasBigChange):
    
    counter += 1
    if counter % 25 == 0:
        print("Iteration ", counter)
        
    hasBigChange = False
    for node in graph.nodes():
        for bid in testBusinessIds:
            bids = graph.nodes[node]["absorbing_bids"]
            isAbsorbant = True if bid in bids else False

            if isAbsorbant:
                continue

            R = 0
            for n in graph.neighbors(node):
                if bid in graph.nodes[n]["r"]:
                    R += graph.nodes[n]["r"][bid]
            R = R/graph.degree(node)

            prevR = R+1
            if bid in graph.nodes[node]["r"]:
                prevR = graph.nodes[node]["r"][bid]
            graph.nodes[node]["r"][bid] = R

            difference = abs(abs(prevR) - abs(R))
            if difference > 0.000001: #10^-6
                hasBigChange = True
        

Iteration  25
Iteration  50
Iteration  75
Iteration  100
Iteration  125
Iteration  150
Iteration  175
Iteration  200


In [14]:
#30m - 0.8988347888436959 - 10^-6
#13m - 0.898927860603693 - 10^-3

#5m - 0.8987000999129854 - 10^-3 ++
#16m - 0.8988345631400708 - 10^-6 ++
testingData = testData.copy()
testingData["predict"] = testingData.apply(lambda x : graph.nodes[x.user_id]["r"][x.business_id], axis=1)
RMSE = sklearn.metrics.mean_squared_error(testingData["stars"], testingData["predict"], squared=False)
RMSE

0.8988345631400708

# Value Propagation - Improved
The algorithm takes a fair bit of time (~25m), so as an improvement, we add another attribute to all nodes.
-"isAbsorbing" dictates as a boolean whether this node is absorbing for the business we are checking.

For this to work, we change the order of our loops, and instead calculate the guess for each business seperatly. This way, we only check whether a node is absorbing for that bussiness on the first iteration, and on next iterations we simply know. This improves the iteration time to ~18m.

In [13]:
#Improved to O(N) complexity for absorbant validation
loadingBar = IntProgress(min=0, max=len(testBusinessIds)) # instantiate the bar
display(loadingBar)

for bid in testBusinessIds:
    
    loadingBar.value += 1
    hasBigChange = True
    firstIteration = True
    
    while(hasBigChange):
        hasBigChange = False
        
        for node in graph.nodes():
            if firstIteration:
                bids = graph.nodes[node]["absorbing_bids"]
                isAbsorbant = True if bid in bids else False
                graph.nodes[node]["isAbsorbant"] = isAbsorbant
            else:
                isAbsorbant = graph.nodes[node]["isAbsorbant"]
            
            if isAbsorbant:
                continue
                
            R = 0
            for n in graph.neighbors(node):
                if bid in graph.nodes[n]["r"]:
                    R += graph.nodes[n]["r"][bid]
            R = R/graph.degree(node)

            prevR = R+1
            if bid in graph.nodes[node]["r"]:
                prevR = graph.nodes[node]["r"][bid]
            graph.nodes[node]["r"][bid] = R

            difference = abs(abs(prevR) - abs(R))
            if difference > 0.000001: #10^-6
                hasBigChange = True
        firstIteration = False

IntProgress(value=0)

# Part 4: Conclusions

| Algorithm   | RMSE  | Time  |
|:----------:|:------:|:------:|
| BA          | 0.867  |   <1s    |
| UCF         | 0.892, K=40| ~1m  |
| Value Prop  | 0.898  |   ~18m    |
| UA          | 0.902  |    <1s   |
| ICF         | 0.906  |   ~2m    |
| SVD         | 3.61, K=100|  ~1m |

By trying all algorithms, we still get the Business Average as the most efficient time and error wise algorithm. The Value Propagation algorithm manages to surpass almost all other algorithms in terms of RMSE, however time is its largest downside. With the hope the VP can be further optimised, which I haven't managed to do, I can see it climbing higher the ranking ladder in terms of time. However the error will hardly be reduced by reducing the threshold further.

**Note: The algorithms where run using the code from exercise 2-2.**